# DNN
fashion MNIST 데이터 셋을 활용해 classification을 해보자.

2️⃣ DNN을 통해 Fashion MNIST Data 분류하기

In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

tensor와 가중치에 대한 연산을 CPU와 GPU 중 어디서 실행할지 결정한다.  
코드 공유 시에 유용하므로 항상 아래 코드를 포함하도록 하자.

cuda인 경우 GPU에서, cpu인 경우는 cpu에서 실행된다.

In [23]:
USE_CUDA = torch.cuda.is_available() # cuda 사용 여부
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

Fashion MNIST는 70,000개로 구성되어 그 수가 많다.  
따라서 Batch size 단위로 나누어 학습시키자.

In [24]:
EPOCHS = 30 # 학습과 평가를 epoch 만큼 진행
BATCH_SIZE = 64

## DNN Structure

### Model class
- torch.nn 상속 Class 정의
    - 생성자에서 각 layer에서 이뤄질 작업을 초기화한다.
    - forward 함수에 forward propagation에 진행될 작업을 정의한다.

In [25]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10) # output layer ; multi-class classification
    
    def forward(self, x):
        x = x.view(-1, 784) # 1차원 행렬로 변환
        x = F.relu(self.fc1(x))        
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

nn.ReLu와 F.relu는 같은 기능을 제공한다. 따라서 원하는 것을 사용해도 된다.   
단, torch.nn.functional은 가중치 없는 연산을, torch.nn 모듈은 가중치 있는 연산을 할 때 사용한다.

#### Model Instance

In [26]:
model = Net().to(DEVICE) # 지정된 장치의 메모리로 전달

In [27]:
optimizer = optim.SGD(model.parameters(), lr=0.01)

### Train 함수 정의
모델의 훈련은 오차를 최소화하기 위해 가중치와 편향을 조정하는 과정이다. Train 함수에 다음의 과정을 정의하자.

In [28]:
def train(model, train_loader, optimizer):
    model.train() # train mode
    
    for batch_idx, (data, target) in enumerate(train_loader):
        # 학습 데이터를 DEVICE의 메모리로 보냄
        data, target = data.to(DEVICE), target.to(DEVICE)
        
        optimizer.zero_grad()
        
        output = model(data)
        
        loss = F.cross_entropy(output, target) # batch size인 64개 loss의 평균값이 return된다.
        loss.backward()
        
        optimizer.step()

- train_loader : 데이터를 공급하는 매개변수  
    (data, target) 형식의 배치를 반환한다.

### Evaluation 함수 정의

#### 모델 성능 평가

- 일반화 성능  
    훈련 데이터 뿐 아니라 모든 데이터에 대해 적용 가능해야 한다.
    
매 epoch가 끝날 때마다 모델을 평가하는 함수를 만들어 보자.

In [29]:
def evaluate(model, test_loader):
    model.eval() # evaluation mode
    
    test_loss = 0
    correct = 0
    
    # 평가 과정에는 기울기 계산이 필요 없다.
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data) #예측 값
            
            # test 오차의 합
            test_loss += F.cross_entropy(output, target,
                                        reduction='sum').item()
            # 모델의 예측이 맞은 개수
            pred = output.max(1, keepdim=True)[1] #예측한 index
            correct += pred.eq(target.view_as(pred)).sum().item()
            
        # 오차 평균
        test_loss /= len(test_loader.dataset)
        #accuracy
        test_accuracy = 100. * correct / len(test_loader.dataset)
            
        return test_loss, test_accuracy
            

- output.max()는 가장 큰 값과, 그 인덱스 2개의 값을 return한다.  
  우리는 모델이 예측한 레이블이 뭔지 궁금한 것이므로 두 번째 값인 인덱스 값을 반환하도록 한다.
  > argmax 함수 : 배열에서 가장 큰 값이 있는 인덱스를 반환하는 함수


- pred.eq(target.view_as(pred)).sum().item()
    - target.view_as(pred) : target을 pred와 형태를 일치시켜 비교하도록 한다.  
    - eq() : target값과 일치하면 1을, 일치하지 않으면 0을 반환하므로 sum()을 통해 정답을 맞춘 개수를 구할 수 있다. 
    
위 정보를 통해 accuracy를 구할 수 있겠다.

## Load Data
신경망 모델을 실행하기 위한 준비는 다 끝났다. 이제 fashionMNIST 데이터를 로드해 실제로 돌려보자.

#### import libraries

In [30]:
from torchvision import datasets, transforms, utils # 이미지 데이터 조작
from torch.utils import data # 데이터 로드 및 조작

In [31]:
# 로드할 data에 적용할 filters
transform = transforms.Compose([
    transforms.ToTensor() # tensor로 변형
])

#### 로드할 data 정의

In [32]:
trainset = datasets.FashionMNIST(
    root = './.data/',
    train = True, # 훈련용 데이터
    download = True, # root 경로에 없으면 download
    transform = transform # load시 적용할 filters
)
testset = datasets.FashionMNIST(
    root = './.data/',
    train = False, # test data
    download = True,
    transform = transform
)

#### Data Load

In [33]:
train_loader = data.DataLoader(
    dataset = trainset,
    batch_size = BATCH_SIZE,
    shuffle = True,
)

test_loader = data.DataLoader(
    dataset = testset,
    batch_size = BATCH_SIZE,
    shuffle = True,
)

아 여기서 의문점이 풀렸다. data를 로드할 당시 batch단위로 로드해왔다. 즉 test_loader 한 단위마다 16개의 이미지가 있는 것이다.

### Run Model

In [34]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {: .4f}, Accuracy: {:.2f}%'.format(
                epoch, test_loss, test_accuracy))

[1] Test Loss:  0.8468, Accuracy: 68.77%
[2] Test Loss:  0.6787, Accuracy: 74.11%
[3] Test Loss:  0.5880, Accuracy: 79.43%
[4] Test Loss:  0.5567, Accuracy: 79.54%
[5] Test Loss:  0.5193, Accuracy: 81.92%
[6] Test Loss:  0.5096, Accuracy: 81.39%
[7] Test Loss:  0.4862, Accuracy: 82.84%
[8] Test Loss:  0.4726, Accuracy: 83.33%
[9] Test Loss:  0.4824, Accuracy: 82.55%
[10] Test Loss:  0.4798, Accuracy: 82.54%
[11] Test Loss:  0.4513, Accuracy: 83.97%
[12] Test Loss:  0.4532, Accuracy: 83.84%
[13] Test Loss:  0.4400, Accuracy: 84.56%
[14] Test Loss:  0.4326, Accuracy: 84.82%
[15] Test Loss:  0.4257, Accuracy: 84.73%
[16] Test Loss:  0.4237, Accuracy: 84.93%
[17] Test Loss:  0.4115, Accuracy: 85.33%
[18] Test Loss:  0.4164, Accuracy: 85.14%
[19] Test Loss:  0.4530, Accuracy: 83.71%
[20] Test Loss:  0.4016, Accuracy: 85.87%
[21] Test Loss:  0.3977, Accuracy: 85.95%
[22] Test Loss:  0.4105, Accuracy: 85.38%
[23] Test Loss:  0.3978, Accuracy: 85.78%
[24] Test Loss:  0.3941, Accuracy: 86.08%
[

----
회고

- 2번째 신경망 모델이여서 pytorch로 어떻게 신경망을 구현하는지 조금 익숙해진 듯하다.
- 학기 중이라 오랜만에 학습했더니, 이전 내용이 또 흐릿하다. 복습해야 겠다.
- batch 학습에 대한 이해가 부족한 것 같다. 추가 학습이 필요해 보인다.
- 다음 CNN을 구현하기 이전에 ANN, DNN에 대한 train, test 함수 구현에 초점을 맞춰 리뷰하고 더불어 MNIST를 통해 pytorch로 data 로드하는 방법을 복습해야 겠다.

### 과적합과 드롭아웃
과적합을 방지하기 위해 아래의 방법을 사용할 수 있다.
>- Early Stopping  
    검증 세트에서의 성능이 나빠지기 직전에 훈련을 중단한다.
>- Data Augmentation
>- Dropout

#### [1] Image Data Augmentation

이미지 데이터는 torchvision에서 제공되는 transforms를 통해 다양한 변주를 주며 데이터 양을 늘릴 수 있다.  
노이즈 추가, 이미지 일부 자르기, 돌리기, 색상 변경 등의 변주를 줄 수 있다.

In [35]:
train_loader = data.DataLoader(
    datasets.FashionMNIST(
        './.data',
        train=True,
        download=True,
        transform = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])),
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_loader = data.DataLoader(
    datasets.FashionMNIST(
        './.data',
        train=False,
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])),
    batch_size=BATCH_SIZE,
    shuffle=True
    
)

- transforms.RandomHorizontalFlip() : 무작위로 수평 뒤집기

#### [2] Dropout
모델의 생성자에 dropout 비율을 지정해 설정할 수 있다.

In [36]:
class Net(nn.Module):
    def __init__(self, dropout_p=0.2):
        super(Net, self).__init__()
        
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10) # output layer ; multi-class classification
        self.dropout_p = dropout_p # 드롭아웃 비율(기본값=0.2)
        
    def forward(self, x):
        x = x.view(-1, 784) # 1차원 행렬로 변환
        x = F.relu(self.fc1(x))     
        # 드롭아웃 추가
        x = F.dropout(x, training=self.training, # 학습모드 여부
                    p=self.dropout_p) # dropout 비율
        x = F.relu(self.fc2(x))
        x = F.dropout(x, training=self.training, 
                    p=self.dropout_p) 
        x = self.fc3(x)
        return x

- forward
    - F.dropout(x, training=self.traininig, p=self.dropout_p)
    - model의 학습모드 여부 및 dropout 비율 전달

In [39]:
model = Net(dropout_p=0.2).to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [40]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {: .4f}, Accuracy: {:.2f}%'.format(
                epoch, test_loss, test_accuracy))

[1] Test Loss:  0.6544, Accuracy: 76.29%
[2] Test Loss:  0.5358, Accuracy: 80.65%
[3] Test Loss:  0.4935, Accuracy: 82.28%
[4] Test Loss:  0.4629, Accuracy: 83.07%
[5] Test Loss:  0.4429, Accuracy: 83.93%
[6] Test Loss:  0.4318, Accuracy: 84.45%
[7] Test Loss:  0.4286, Accuracy: 84.31%
[8] Test Loss:  0.4092, Accuracy: 85.20%
[9] Test Loss:  0.3999, Accuracy: 85.47%
[10] Test Loss:  0.3925, Accuracy: 85.84%
[11] Test Loss:  0.3844, Accuracy: 86.16%
[12] Test Loss:  0.3824, Accuracy: 86.14%
[13] Test Loss:  0.3704, Accuracy: 86.73%
[14] Test Loss:  0.3736, Accuracy: 86.57%
[15] Test Loss:  0.3700, Accuracy: 86.66%
[16] Test Loss:  0.3602, Accuracy: 86.94%
[17] Test Loss:  0.3600, Accuracy: 87.07%
[18] Test Loss:  0.3585, Accuracy: 87.02%
[19] Test Loss:  0.3542, Accuracy: 87.01%
[20] Test Loss:  0.3458, Accuracy: 87.47%
[21] Test Loss:  0.3430, Accuracy: 87.52%
[22] Test Loss:  0.3492, Accuracy: 87.44%
[23] Test Loss:  0.3429, Accuracy: 87.61%
[24] Test Loss:  0.3424, Accuracy: 87.78%
[